In [ ]:
from collections import defaultdict, Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import entropy as compute_entropy
import json
import argparse
import scipy.stats
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import OlmoeForCausalLM, AutoTokenizer, AutoModelForCausalLM
import torch
import time
import pickle as pkl
import json
import numpy as np
from collections import defaultdict, Counter
from tqdm import tqdm
from pathlib import Path
import argparse
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import math
from utils import *

In [ ]:
model, tokenizer = load_model("/home/mila/k/khandela/scratch/ai2-llm/checkpoints/OLMoE/multilingual-5langs-lbl-1500",0)
layers_to_analyze = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
languages = ["en", "hi", "ar", "ru", "zh"]


In [ ]:
all_lang_expert_counts = defaultdict(lambda: defaultdict(Counter))
all_lang_token_assignments = defaultdict(lambda: defaultdict(lambda: defaultdict(Counter)))
position_based_counts_all = defaultdict(lambda: defaultdict(lambda: defaultdict(Counter)))
coactivation_counts_all = defaultdict(lambda: defaultdict(Counter))

In [ ]:
for lang in languages:
    file_path = f"/home/mila/k/khandela/OLMoE/scripts/data/validation/{lang}.txt"
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read().replace("\n", " ")

    # Run the analysis for the current language
    layer_counts, token_map = collect_language_expert_stats(text, layers_to_analyze, model, tokenizer)

    for layer in layers_to_analyze:
        all_lang_expert_counts[lang][layer].update(layer_counts[layer])
        for expert_id, tokens in token_map[layer].items():
            all_lang_token_assignments[lang][layer][expert_id].update(tokens)

In [ ]:
entropies, kls = plot_entropy_kl_summary(all_lang_expert_counts, layers_to_analyze)
plot_kl_heatmaps(all_lang_expert_counts, layers_to_analyze,languages)

In [ ]:
get_shared_and_unique_experts(all_lang_expert_counts, layers_to_analyze, 0.05)

In [ ]:
get_shared_and_unique_experts_decoded(all_lang_expert_counts, all_lang_token_assignments, layers_to_analyze, tokenizer, 0.05, 100)

In [ ]:
plot_all_langs_expert_token_counts(all_lang_expert_counts,layers_to_analyze)

In [ ]:
# model, tokenizer = load_model("/home/mila/k/khandela/scratch/ai2-llm/checkpoints/OLMoE/multilingual-5langs-nolbl-1900",0)
model, tokenizer = load_model("allenai/OLMoE-1B-7B-0924-Instruct",0)
layers_to_analyze = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
languages = ["en", "hi", "ar", "ru", "zh"]


In [ ]:
from collections import defaultdict, Counter
import json

# Initialize data structures
all_lang_expert_counts = defaultdict(lambda: defaultdict(Counter))
all_lang_token_assignments = defaultdict(lambda: defaultdict(lambda: defaultdict(Counter)))
position_based_counts_all = defaultdict(lambda: defaultdict(lambda: defaultdict(Counter)))
coactivation_counts_all = defaultdict(lambda: defaultdict(Counter))


jsonl_file_path = "/home/mila/k/khandela/OLMoE/global_mmlu_multilingual_subjects.jsonl"

for lang in languages:
    with open(jsonl_file_path, "r", encoding="utf-8") as f:
        for line in f:
            entry = json.loads(line)
            for category, translations in entry.items():
                sentence = translations.get(lang)
                if not sentence:
                    continue
                
                layer_counts, token_map = collect_language_expert_stats(sentence, layers_to_analyze, model, tokenizer)

                for layer in layers_to_analyze:
                    all_lang_expert_counts[lang][layer].update(layer_counts[layer])
                    for expert_id, tokens in token_map[layer].items():
                        all_lang_token_assignments[lang][layer][expert_id].update(tokens)




In [ ]:
plot_kl_heatmaps(all_lang_expert_counts, layers_to_analyze,languages)
entropies, kls = plot_entropy_kl_summary(all_lang_expert_counts, layers_to_analyze)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# Example: Pre-assign colors to languages
color_map = {
    'en': 'tab:blue',
    'hi': 'tab:orange',
    'zh': 'tab:red',
    'ar': 'tab:green',
    'ru' : 'tab:purple',
    # add more languages if needed
}

# (Optional) if you want to auto-assign colors dynamically
# colors = cm.get_cmap('tab10', len(languages))
# color_map = {lang: colors(i) for i, lang in enumerate(languages)}
plt.figure(figsize=(10, 4))
for lang in languages:
    color = color_map.get(lang, 'black')
    plt.plot(layers_to_analyze, entropies[lang], marker='o', label=lang.upper(), color=color, linestyle='-')
    plt.plot(layers_to_analyze, entropies_nolbl[lang], marker='o', label=f"{lang.upper()} (No LBL)", color=color, linestyle='--', alpha=0.7)
    plt.plot(layers_to_analyze, entropies_lbl[lang], marker='o', label=f"{lang.upper()} (LBL)", color=color, linestyle=':', alpha=0.7)
plt.title("Entropy across Layers")
plt.xlabel("Layer")
plt.ylabel("Entropy")
plt.grid(True)
plt.tight_layout()

# Put legend outside
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
plt.savefig("analysis/entropy.png", bbox_inches='tight')
print("Entropy across Layers saved at analysis/entropy.png")


# --- Plot KL Divergence ---
plt.figure(figsize=(10, 4))
for lang in languages:
    color = color_map.get(lang, 'black')
    plt.plot(layers_to_analyze, kls[lang], marker='o', label=lang.upper(), color=color, linestyle='-')
    plt.plot(layers_to_analyze, kls_nolbl[lang], marker='o', label=f"{lang.upper()} (No LBL)", color=color, linestyle='--', alpha=0.7)
    plt.plot(layers_to_analyze, kls_lbl[lang], marker='o', label=f"{lang.upper()} (LBL)", color=color, linestyle=':', alpha=0.7)
plt.title("KL Divergence from Uniform across Layers")
plt.xlabel("Layer")
plt.ylabel("KL Divergence")
plt.grid(True)
plt.tight_layout()

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
plt.savefig("analysis/kl_divergence.png", bbox_inches='tight')
print("KL Divergence from Uniform across Layers saved at analysis/kl_divergence.png")
